In [79]:
import pandas as pd
import numpy as np
import sklearn, xgboost
import warnings
warnings.filterwarnings('ignore')

train_df = pd.read_csv('304_travel_insurance_train.csv')
test_df = pd.read_csv('304_travel_insurance_test.csv')

In [80]:
y_train = train_df['TravelInsurance']
X_train = train_df.drop(['TravelInsurance'], axis = 1)

In [81]:
X_test = test_df

In [82]:
print(X_train.head(3))

     ID  Age               Employment Type GraduateOrNot  AnnualIncome  \
0  1704   26  Private Sector/Self Employed           Yes       1400000   
1   491   28  Private Sector/Self Employed           Yes       1100000   
2   414   33  Private Sector/Self Employed           Yes       1400000   

   FamilyMembers  ChronicDiseases FrequentFlyer EverTravelledAbroad  
0              3                1            No                 Yes  
1              4                1            No                  No  
2              4                0            No                 Yes  


In [83]:
print(X_train['Employment Type'].unique())

['Private Sector/Self Employed' 'Government Sector']


In [84]:
# 삭제 : ID
# 건들 ㄴ : ['FamilyMembers', 'ChronicDiseases']
# 더미 : ['Employment Type', 'GraduateOrNot', 'FrequentFlyer', 'EverTravelledAbroad']
# 라벨 인코더 : 
# 스케일링 : ['Age', 'AnnualIncome'] 

In [85]:
COL = ['FamilyMembers', 'ChronicDiseases']
COL_DUM = ['Employment Type', 'GraduateOrNot', 'FrequentFlyer', 'EverTravelledAbroad']
COL_NUM = ['Age', 'AnnualIncome', 'Income/family'] 
# 불필요 피쳐 삭제
X_train = X_train.drop(['ID'], axis = 1)
X_test = X_test.drop(['ID'], axis = 1)

In [86]:
# 더미변수 처리
X_train_dum = pd.get_dummies(X_train[COL_DUM])
X_test_dum = pd.get_dummies(X_test[COL_DUM])

In [87]:
X_train = X_train.drop(COL_DUM, axis = 1)
X_train = pd.concat([X_train, X_train_dum], axis = 1)
X_test = X_test.drop(COL_DUM, axis = 1)
X_test = pd.concat([X_test, X_test_dum], axis = 1)

In [88]:
X_train['Income/family'] = X_train['AnnualIncome'] / X_train['FamilyMembers']
X_test['Income/family'] = X_test['AnnualIncome'] / X_test['FamilyMembers']

In [89]:
# train test split
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size = 0.3)

In [90]:
# MinMaxScale
from sklearn.preprocessing import MinMaxScaler

mms = MinMaxScaler()
mms.fit(X_tr[COL_NUM])
X_tr[COL_NUM] = mms.fit_transform(X_tr[COL_NUM])
X_test[COL_NUM] = mms.fit_transform(X_test[COL_NUM])
X_val[COL_NUM] = mms.fit_transform(X_val[COL_NUM])

In [61]:
# StandardScaler
#from sklearn.preprocessing import StandardScaler

#scaler = StandardScaler()
#scaler.fit(X_tr[COL_NUM])
#X_tr[COL_NUM] = scaler.fit_transform(X_tr[COL_NUM])
#X_test[COL_NUM] = scaler.fit_transform(X_test[COL_NUM])
#X_val[COL_NUM] = scaler.fit_transform(X_val[COL_NUM])

In [91]:
from xgboost import XGBClassifier
model_xgb = XGBClassifier(random_state = 22)
model_xgb.fit(X_tr, y_tr.values.ravel())

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=22,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method=None, validate_parameters=False, verbosity=None)

In [92]:
from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier(random_state = 11)
model_rf.fit(X_tr, y_tr.values.ravel())

RandomForestClassifier(random_state=11)

In [93]:
from sklearn.metrics import roc_auc_score

pred_xgb = model_xgb.predict_proba(X_val)
pred_rf = model_rf.predict_proba(X_val)

xgb_score = roc_auc_score(y_val, pred_xgb[:, 1])
rf_score = roc_auc_score(y_val, pred_rf[:, 1])

In [94]:
print(xgb_score, rf_score) # Min-MAx : 성능 더 훌륭

0.8275509769699034 0.8310203907879614


In [113]:
from xgboost import XGBClassifier
model_xgb = XGBClassifier(learning_rate = 0.15, max_depth = 7,  colsample_bytree=0.4)
model_xgb.fit(X_tr, y_tr.values.ravel())

from sklearn.metrics import roc_auc_score

pred_xgb = model_xgb.predict_proba(X_val)

xgb_score = roc_auc_score(y_val, pred_xgb[:, 1])

print(xgb_score)

0.840790089821634
